# Sistemas Inteligentes

## Practica 1: Búsqueda en espacio de estados

### Declaracion e implementacion de clases necesarias

In [2]:

class Estado:
    def __init__(self, valor):
          self.valor = valor

     def __eq__(self, other):
        return isinstance(other, Estado) and self.valor == other.valor

     def __hash__(self):
        return hash(self.valor)

class Accion:
    def __init__(self, descripcion):
        self.descripcion = descripcion

class Nodo:
    def __init__(self, estado, padre=None, accion=None):
        self.estado = estado
        self.padre = padre
        self.accion = accion

    def hijo(self, accion):
        nuevo_estado = self.estado.resultado(accion)
        return Nodo(nuevo_estado, self, accion)

    def solucion(self):
        acciones = []
        nodo = self
        while nodo.padre is not None:
            acciones.append(nodo.accion)
            nodo = nodo.padre
        acciones.reverse()
        return acciones

class Problema:
    def __init__(self, estado_inicial, estado_objetivo):
        self.estado_inicial = estado_inicial
        self.estado_objetivo = estado_objetivo

    def es_objetivo(self, estado):
        return estado == self.estado_objetivo

    def acciones(self, estado):
        raise NotImplementedError

    def resultado(self, estado, accion):
        raise NotImplementedError
    

IndentationError: unindent does not match any outer indentation level (<string>, line 5)

### 1: Estrategia de búsqueda no informada

#### 1.1: Búsqueda primero en anchura

In [4]:
def busqueda_en_arbol(problema, frontera):
    # Insertar el nodo inicial en la frontera
    nodo_inicial = Nodo(problema.estado_inicial)
    frontera.append(nodo_inicial)

    while True:
        # Si la frontera está vacía, devolver FALLO
        if not frontera:
            return None
        
        # Borrar el primer nodo de la frontera
        nodo = frontera.pop(0)
        
        # Si el nodo contiene un estado objetivo, devolver la solución
        if problema.es_objetivo(nodo.estado):
            return nodo.solucion()
        
        # Expandir el nodo y añadir los hijos a la frontera
        for accion in problema.acciones(nodo.estado):
            hijo = nodo.hijo(accion)
            frontera.append(hijo)

In [5]:
def expandir(nodo, problema):
    sucesores = []
    for accion in problema.acciones(nodo.estado):
        resultado = problema.resultado(nodo.estado, accion)
        s = Nodo(resultado, nodo, accion)
        s.coste = nodo.coste + problema.coste_individual(nodo, accion, s)
        s.profundidad = nodo.profundidad + 1
        sucesores.append(s)
    return sucesores